In [1]:
import numpy as np

class NeuralNetworkLayer(object):
    '''
    A Neural Network Layer
    '''
    def __init__(self, input_size, output_size, name=None):
        self.number_of_neurons = output_size
        self.inputs_per_neuron = input_size
        self.synaptic_weights = 2 * np.random.random((input_size, output_size)) - 1
        self.name = name
    
    def __repr__(self):
        return str({
            'number_of_neurons': self.number_of_neurons, 
            'inputs_per_neuron': self.inputs_per_neuron, 
            'synaptic_weights': self.synaptic_weights, 
            'name': self.name})

class MultiLayerNeuralNetwork(object):
    '''
    A MultiLayer Neural Network
    '''
    def __init__(self, layer_sizes, name=None):
        self.layers = [ NeuralNetworkLayer(input_size, output_size, name='layer_{}'.format(i)) 
                       for (i, (input_size, output_size)) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])) ]
        self.name = name

    def __sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def __sigmoid_derivative(self, x):
        return x * (1 - x)

    def predict(self, inputs):
        '''
        Returns the outputs of all layers. Note that:
            - Outputs of last layer is the actual prediction
            - Outputs of the other layers are still useful (and necessary when training)
        '''
        layer_outputs_stack = []
        layer_inputs = inputs # layer_inputs for first layer
        for layer in self.layers:
            layer_outputs = self.__sigmoid(np.dot(layer_inputs, layer.synaptic_weights))
            layer_outputs_stack.append(layer_outputs)
            layer_inputs = layer_outputs # layer_inputs for next layer
        return layer_outputs_stack
    
    def train_step(self, training_set_inputs, training_set_outputs):
        '''
        Training function
        '''
        # Get the outputs for all layers
        layer_outputs_stack = self.predict(training_set_inputs)
        last_layer_outputs = layer_outputs_stack[-1]
        # Now, for each layer, we calculate errors and backpropagate
        reversed_layer_outputs_stack = list(reversed(layer_outputs_stack))
        reversed_layer_inputs_stack = reversed_layer_outputs_stack[1:] + [training_set_inputs]
        reversed_layer_stack = list(reversed(self.layers))
        # Calculate the error for the layer in the stack
        layer_errors = training_set_outputs - last_layer_outputs # layer_errors for last layer
        for (layer_outputs, layer_inputs, layer) in zip(reversed_layer_outputs_stack, 
                                                        reversed_layer_inputs_stack, 
                                                        reversed_layer_stack):
            # Calculate the deltas for this layer
            layer_deltas = layer_errors * self.__sigmoid_derivative(layer_outputs)
            # Calculate the weight adjustments for this layer
            layer_adjustments = layer_inputs.T.dot(layer_deltas)
            # Calculate the errors for the next layer in the stack 
            layer_errors = layer_deltas.dot(layer.synaptic_weights.T)
            # Adjust the weights for this layer
            layer.synaptic_weights += layer_adjustments
        # Returning the prediction, so it can be used to calculate loss or accuracy at each training step
        return last_layer_outputs
    
    def __repr__(self):
        return str({'layers': self.layers, 
                    'name': self.name})

In [2]:
np.random.seed(1)
model = MultiLayerNeuralNetwork([3, 4, 3, 1], name='mlnn')
print(model)

{'layers': [{'synaptic_weights': array([[-0.16595599,  0.44064899, -0.99977125, -0.39533485],
       [-0.70648822, -0.81532281, -0.62747958, -0.30887855],
       [-0.20646505,  0.07763347, -0.16161097,  0.370439  ]]), 'number_of_neurons': 4, 'inputs_per_neuron': 3, 'name': 'layer_0'}, {'synaptic_weights': array([[-0.5910955 ,  0.75623487, -0.94522481],
       [ 0.34093502, -0.1653904 ,  0.11737966],
       [-0.71922612, -0.60379702,  0.60148914],
       [ 0.93652315, -0.37315164,  0.38464523]]), 'number_of_neurons': 3, 'inputs_per_neuron': 4, 'name': 'layer_1'}, {'synaptic_weights': array([[ 0.7527783 ],
       [ 0.78921333],
       [-0.82991158]]), 'number_of_neurons': 1, 'inputs_per_neuron': 3, 'name': 'layer_2'}], 'name': 'mlnn'}


In [3]:
# The training set. We have 7 examples, each consisting of 3 input values and 1 output value
training_set_inputs = np.array([[0, 0, 1], [0, 1, 1], [1, 0, 1], [0, 1, 0], [1, 0, 0], [1, 1, 1], [0, 0, 0]])
training_set_outputs = np.array([[0, 1, 1, 1, 1, 0, 0]]).T
# Train the neural network using the training set
for iteration in range(100000):
    pred_outputs = model.train_step(training_set_inputs, training_set_outputs)
    if(iteration % 1000 == 0):
        errors = training_set_outputs - pred_outputs
        mse = (errors ** 2).mean()
        print('mse: {}'.format(mse))
print('Finished training!')
# Model after training:
print(model)

mse: 0.24266088222915755
mse: 0.00039285198274240405
mse: 0.00014449637024398454
mse: 8.63737827340187e-05
mse: 6.1030231478078415e-05
mse: 4.696109875877256e-05
mse: 3.8054028963115765e-05
mse: 3.192624864700634e-05
mse: 2.7461233667636635e-05
mse: 2.4067820709852033e-05
mse: 2.140437023177554e-05
mse: 1.9259969274216807e-05
mse: 1.749750600631987e-05
mse: 1.6024062329174324e-05
mse: 1.477447889635889e-05
mse: 1.3701734918804264e-05
mse: 1.2771062898339076e-05
mse: 1.1956211359611484e-05
mse: 1.1236994477714822e-05
mse: 1.0597640298261893e-05
mse: 1.0025649919993934e-05
mse: 9.51099250351692e-06
mse: 9.045526308916226e-06
mse: 8.62257512210963e-06
mse: 8.236613558562766e-06
mse: 7.883029976212709e-06
mse: 7.557945577858168e-06
mse: 7.258074776804811e-06
mse: 6.980616260966469e-06
mse: 6.723167169926439e-06
mse: 6.483654866483043e-06
mse: 6.260282238897176e-06
mse: 6.051483507452444e-06
mse: 5.855888257843095e-06
mse: 5.672291970737802e-06
mse: 5.499631720396973e-06
mse: 5.336966015996

In [4]:
# Just checking how it adjusts to the training set
layer_outputs_stack = model.predict(training_set_inputs)
print(layer_outputs_stack[len(layer_outputs_stack) - 1])

[[  3.33120340e-04]
 [  9.98677044e-01]
 [  9.98688527e-01]
 [  9.98841625e-01]
 [  9.98953427e-01]
 [  1.90535554e-03]
 [  1.74481021e-03]]


In [5]:
# Test the neural network with a new input -> ?:
test_set_inputs = np.array([[1, 1, 0]])
layer_outputs_stack = model.predict(test_set_inputs)
print(layer_outputs_stack[-1])

[[ 0.00212492]]
